In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Read and merge Census

In [2]:
data_path = "../data/data_2017/"
acs1 = pd.read_csv(data_path + 'ACSDP5Y2017.DP03_data_with_overlays_2022-03-17T161301.csv', na_values=['(X)']) #, encoding='cp1252'
acs2 = pd.read_csv(data_path + 'ACSDP5Y2017.DP05_data_with_overlays_2022-03-01T200530.csv', na_values=['(X)']) 


#DP03_0095E - health care coverage

def remove_cols(df):
    
    #save county
    county = df['GEO_ID']
    county = county.str[-5:]
    county = county.astype(str) 
    county = county.loc[1:]
    
    #find the bad columns
    cols = list(df.columns)
    trim_cols1 = []
    for col in cols:
        if 'PE' in col or 'DP03_0095E' in col:
            trim_cols1.append(col)

    df = df[trim_cols1]
    df = df.loc[1:]
    df.fillna(0, inplace = True)
    
    
    #filter out bad columns
    df['County'] = county
    
    return df

dfs = []

for i in [acs1,acs2]:
    dfs.append(remove_cols(i))

for i in dfs:
    print(i.shape)

(3220, 139)
(3220, 90)


In [3]:
from functools import partial, reduce
[acs1, acs2] = dfs
merge = partial(pd.merge, on=['County'], how='outer')
all_acs = reduce(merge, dfs)
all_acs

,DP03_0001PE,DP03_0002PE,DP03_0003PE,DP03_0004PE,DP03_0005PE,DP03_0006PE,DP03_0007PE,DP03_0008PE,DP03_0009PE,DP03_0010PE,...,DP05_0080PE,DP05_0081PE,DP05_0082PE,DP05_0083PE,DP05_0084PE,DP05_0085PE,DP05_0086PE,DP05_0087PE,DP05_0088PE,DP05_0089PE
0,42974,60.3,59.2,56.1,3.1,1.1,39.7,25436,5.2,22202,...,0.9,0.0,0.1,1.6,0.0,1.6,0,41016,47.9,52.1
1,163759,58.0,57.9,54.7,3.2,0.1,42.0,94742,5.5,84553,...,0.7,0.0,0.2,1.4,0.3,1.1,0,155376,48.1,51.9
2,21241,47.7,47.7,41.8,5.9,0.0,52.3,10139,12.4,9853,...,0.6,0.0,0.7,0.8,0.0,0.8,0,20269,53.5,46.5
3,18415,48.3,48.3,44.4,4.0,0.0,51.7,8899,8.2,8462,...,0.0,0.0,0.0,0.7,0.0,0.7,0,17662,53.9,46.1
4,45924,49.0,48.9,46.6,2.4,0.1,51.0,22471,4.9,23323,...,0.1,0.0,0.2,1.4,0.0,1.4,0,42513,48.7,51.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,44583,38.4,38.4,31.9,6.4,0.0,61.6,17098,16.8,23602,...,0.0,0.0,0.1,0.0,0.0,0.0,0,42838,47.0,53.0
3216,7136,47.0,47.0,41.0,6.0,0.0,53.0,3355,12.8,3573,...,0.0,0.0,0.0,0.3,0.0,0.3,0,7045,49.8,50.2
3217,18999,48.2,48.1,36.2,11.9,0.1,51.8,9141,24.8,9885,...,0.0,0.0,0.0,0.0,0.0,0.0,0,18053,47.6,52.4
3218,28515,37.1,37.0,27.6,9.4,0.1,62.9,10563,25.4,14820,...,0.0,0.0,0.0,0.0,0.0,0.0,0,27523,47.6,52.4


In [4]:
all_acs.to_csv('../data/processed_data/county_characteristics_2017.csv', index=False)